In [1]:
"""
snipper.io/s/-MSTCnlcnC3p7aUSnOm_
https://www.kaggle.com/c/clickbait-news-detection/

"""

'\nsnipper.io/s/-MSTCnlcnC3p7aUSnOm_\nhttps://www.kaggle.com/c/clickbait-news-detection/\n\n'

In [2]:
from copy import deepcopy

from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram, HashingTF, IDF, CountVectorizer, OneHotEncoder, StringIndexer, Word2Vec
from pyspark.sql.functions import col, udf, length, avg, lit, concat, size, array
from pyspark.sql.types import IntegerType, StringType, ArrayType
#from pyspark.mllib.classification import NaiveBayes#, NaiveBayesModel
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession

import re
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
#import nltk
#nltk.download('wordnet')

In [3]:
try:
    sc.stop()
except:
    print("no sc to stop")
    
# try:
#     spark.stop()
# except Exception as e:
#     print("{}, {}".format(e,type(e)))
    

In [4]:
conf = SparkConf().setAppName('clickbait').setMaster('local[2]')
sc = SparkContext.getOrCreate(conf = conf)

In [5]:
spark = SparkSession.builder.appName('ClickBait')\
    .config("spark.driver.memory", "8g")\
    .getOrCreate()

In [6]:
spark

In [7]:
# pyspark_stopwords = StopWordsRemover().getStopWords()#https://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.feature.StopWordsRemover
# display(type(pyspark_stopwords), len(pyspark_stopwords), pyspark_stopwords)


In [8]:
train_path = "file:///home/hadoop/data/clickbait/train.csv"
#train_path = "/data/train.csv"
#train_path = "hdfs://localhost:9000/data/train.csv;"


In [9]:
# load main training dataframe

t_df = spark.read.csv(train_path, header=True, inferSchema=True, encoding="utf-8")
#t_df=t_df.sample(False, 0.1, seed=0).select("*").limit(1600)
t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [10]:
t_df.count()

26568

In [11]:
t_df=t_df.na.drop()

t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [12]:
t_df.count()

19658

In [13]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)



In [14]:
#from string import punctuation

def strip_possessive(l_):
    ptrn=re.compile(r"['`’]s", flags=re.I)
    r__ = [ptrn.sub('', s_) for s_ in l_]
    return(r__)  

@udf("string")
def strip_possessive_str(s_):
    ptrn=re.compile(r"['`’]s", flags=re.I)
    r__ = ptrn.sub(' ', s_)
    return(r__)


###
def strip_func(l_):
    ptrn = re.compile('[\W_]+')
    r__ = [ptrn.sub('', s_) for s_ in l_]
    return(r__)

@udf("string")
def strip_func_str(s_):
    ptrn = re.compile('[\W_]+')
    r__ = ptrn.sub(' ', s_)
    return(r__)

#udf_strip_func = udf(strip_func, ArrayType(StringType()))

# def strip_pos_spchars(l_):
#     ptrn_pos = re.compile(r"[']s", flags=re.I)
#     ptrn_spchars = re.compile('[\W_]+')
#     r__ = [ptrn.sub('', s_) for s_ in l_]
#     [ptrn_spchars.sub('', ptrn_pos.sub('', s_)) for s_ in l_]
#     return(r__)     
#x = list(one_sss_list.take(1)[0].asDict()['sss'])

#x = ["World's fair", "Biden's administration", "fergi's song", "Xi's policy"]

#x, strip_func(x), strip_possessive(x), #strip_possessive_str("World's fair ... Biden's administration, Fergi's song. Xi's policy")
#strip_func(one_sss_list.take(1)[0].asDict()['sss'])

In [15]:
help(udf)

Help on function udf in module pyspark.sql.functions:

udf(f=None, returnType=StringType)
    Creates a user defined function (UDF).
    
    .. note:: The user-defined functions are considered deterministic by default. Due to
        optimization, duplicate invocations may be eliminated or the function may even be invoked
        more times than it is present in the query. If your function is not deterministic, call
        `asNondeterministic` on the user defined function. E.g.:
    
    >>> from pyspark.sql.types import IntegerType
    >>> import random
    >>> random_udf = udf(lambda: int(random.random() * 100), IntegerType()).asNondeterministic()
    
    .. note:: The user-defined functions do not support conditional expressions or short circuiting
        in boolean expressions and it ends up with being executed all internally. If the functions
        can fail on special rows, the workaround is to incorporate the condition into the functions.
    
    .. note:: The user-defined

In [16]:
stemmer=SnowballStemmer("english")

lemmatizer = WordNetLemmatizer()


def stem_lem_per_word(word, pos='n'):
    sl_word = stemmer.stem(word)
    sl_word = lemmatizer.lemmatize(sl_word, pos=pos)
    return(sl_word)
    #pos=noun
    

def stem_lem_list(list_of_words, pos='n'):
    # l_ is supposed to be a  column value but nltk stemmers are expecting just a python list?
    r_list = [stem_lem_per_word(w_) for w_ in list_of_words]
    r_list = [w_ for w_ in r_list if w_ not in [""]]
    return(r_list)

@udf("array<string>")
def stem_lem(arr_str):
#    pass
     return stem_lem_list(arr_str)



st_cases=["writing", "write", "wrote", "writing", "written", "drag", "dragged", "drug", "dragon", ""]
[stem_lem_per_word(case) for case in st_cases]
#t_df.select("s3").map(stem_lem)
stem_lem_list(st_cases)


['write',
 'write',
 'wrote',
 'write',
 'written',
 'drag',
 'drag',
 'drug',
 'dragon']

In [17]:

# def stem_lem(w_, pos='n'):
#     return(lemmatizer.lemmatize(stemmer.stem(w_), pos=pos))

# stem_lem_cases= ["writing", "write", "wrote", "writing", "written", "drag", "dragged", "drug", "dragon"]

# display(
#     stem_lem_cases,
#     [stemmer.stem(w) for w in stem_lem_cases],
#     [lemmatizer.lemmatize(w, pos='n') for w in stem_lem_cases],
#     [stem_lem(w) for w in stem_lem_cases]
#     )

#stem_lem = udf(stem_lem, StringType())

In [18]:
t_df.columns, t_df

(['id', 'title', 'text', 'label'],
 DataFrame[id: string, title: string, text: string, label: string])

In [19]:
t_df.select('title','text','label').limit(20).show()


+--------------------+--------------------+--------------------+
|               title|                text|               label|
+--------------------+--------------------+--------------------+
|China and Economi...|Economists genera...|                news|
|Trade to Be a Big...|LONDON—British Pr...|                news|
|The Top Beaches I...|Beaches come in a...|           clickbait|
|Sheriff’s Report ...|A timeline of wha...|           clickbait|
|Surgeon claiming ...|An Italian neuros...|                news|
|This Is How Diffe...|Spanx does not do...|                news|
|Samantha Bee and ...|“Jane Pauley! You...|                news|
|Krauthammer: Syri...|"Charles Krautham...|"" in that he wou...|
|Rust Belt voters ...|"The United State...|  "" says Matt Seely|
|As Illegal Outpos...|"As Jewish settle...|         for Zionism|
|$94.4m $198m $112...|In the very best ...|                news|
|Trump’s Mar-a-Lag...|"President Donald...|  some significantly|
|STOCKS SLIP TO EN...|"Bo

In [20]:
t_df=t_df.withColumn("s", 
                strip_func_str(strip_possessive_str(
                    concat(col('title'),col("text")))
                              ))

In [21]:
#[strip_func_str(c__).show() for c__ in ["s"]]
#dir(strip_func_str("s").get)

In [22]:
#t_df.select('title','text','label').limit(20).show()
t_df['text']#.limit(20).show()


Column<b'text'>

In [23]:
t_df = t_df.withColumn("s", strip_func_str(strip_possessive_str(
    concat(col('title'),col("text")))))

In [24]:
#t_df = t_df.withColumn("s", strip_func_str(strip_possessive_str(col('s'))))


t_df.select('s').show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
concat, col

(<function pyspark.sql.functions.concat(*cols)>,
 <function pyspark.sql.functions._create_function.<locals>._(col)>)

In [26]:
t_df.columns

['id', 'title', 'text', 'label', 's']

In [27]:
"’"=="`"

False

In [28]:
selection = ['s', "title", "text"]

display("avg string length of columns")

t_df \
    .select(*(length(col(c)).alias(c) for c in selection)) \
    .agg(*(avg(col(c)).alias(c) for c in selection)).show()

# t_df \
#     .select(*(size(col(c)).alias(c) for c in selection)) \
#     .agg(*(avg(col(c)).alias(c) for c in selection)).show()


'avg string length of columns'

+------------------+-----------------+-----------------+
|                 s|            title|             text|
+------------------+-----------------+-----------------+
|2589.0048835079865|72.96103367585715|2600.035201953403|
+------------------+-----------------+-----------------+



In [29]:
from collections.abc import Iterable

def get_avg_len_cols(df, selection):
    """for str length in pyspark columns"""
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(length(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

def get_avg_size_cols(df, selection):
    """for arrays of str in pyspark df"""
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(size(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

In [30]:
t_df.columns
#get_avg_size_cols

['id', 'title', 'text', 'label', 's']

In [31]:
get_avg_len_cols(t_df, ["title", "text", "s"])

+-----------------+-----------------+------------------+
|            title|             text|                 s|
+-----------------+-----------------+------------------+
|72.96103367585715|2600.035201953403|2589.0048835079865|
+-----------------+-----------------+------------------+



In [32]:
t_df = Tokenizer(inputCol = "s", outputCol = "s1").transform(t_df) # first text transformation is pyspark builtin tokenizer
#some punctuation like ',' is still showing after Tokenizer().transform. ??

In [33]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [34]:
t_df.select("s1").limit(5).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
t_df = StopWordsRemover(inputCol = "s1", outputCol="s2").transform(t_df)



In [36]:
get_avg_size_cols(t_df, ['s1','s2'])

+-----------------+------------------+
|               s1|                s2|
+-----------------+------------------+
|453.1628853393021|262.91845559059925|
+-----------------+------------------+



In [37]:
# get_avg_size_cols(t_df, ['sss'])

In [38]:
# t_df.select('sss').limit(5).show(truncate=False)

In [39]:
# t_df.sample(False, 0.1, seed=0).limit(1).sss.getItem()

In [40]:
t_df.sample(False, 0.1, seed=0).select("s2").limit(1).show(truncate=False)

my_sample=t_df.sample(False, 0.1, seed=0).select("s2").limit(1)


+--------------------------------------------------------------------------------------------------+
|s2                                                                                                |
+--------------------------------------------------------------------------------------------------+
|[, asap, ferg, remy, ma, rep, east, coast, new, song, asap, ferg, repping, east, coast, new, song]|
+--------------------------------------------------------------------------------------------------+



In [41]:
my_sample.show()
my_sample.stat

#from pyspark.sql.functions import Row
dir(my_sample.collect()[0])
#stem_lem(one_sss_list)

#stem_lem(my_sample.collect()[0].asDict()['s2'])

stem_lem_list(my_sample.collect()[0].asDict()['s2'])


stem_lem_list(t_df.select("s2").collect()[0].asDict()['s2'])

#t_df = df.withColumn("s3", stem_lem("s2"))

#my_sample.collect()[0].asDict()['s2']

# #dir(one_sss_list),
# dir(one_sss_list.take(1)[0])
# one_sss_list.take(1)[0].asDict()['sss']
#stem_lem(t_df.sample(False, 0.1, seed=0).select("sss").limit(1))

+--------------------+
|                  s2|
+--------------------+
|[, asap, ferg, re...|
+--------------------+



['china',
 'econom',
 'reform',
 'xi',
 'jinp',
 'track',
 'record',
 'economist',
 'general',
 'agre',
 'china',
 'must',
 'overhaul',
 'huge',
 'wast',
 'economi',
 'want',
 'continu',
 'grow',
 'year',
 'come',
 'mean',
 'limit',
 'polit',
 'interfer',
 'bank',
 'financi',
 'system',
 'make',
 'bloat',
 'industri',
 'sensit',
 'market',
 'forc',
 'lower',
 'barrier',
 'foreign',
 'trade',
 'invest',
 'propon',
 'took',
 'heart',
 'late',
 '2012',
 'presid',
 'xi',
 'jinp',
 'took',
 'formal',
 'control',
 'chine',
 'communist',
 'parti',
 'pledg',
 'crack',
 'corrupt',
 'cut',
 'red',
 'tape',
 'today',
 'mr',
 'xi',
 'near',
 'end',
 'first',
 'five',
 'year',
 'term',
 'much',
 'optim',
 'among',
 'economist',
 'fade',
 'china',
 'remain',
 'heavili',
 'depend',
 'larg',
 'invest',
 'maintain',
 'brisk',
 'slow',
 'econom',
 'growth',
 'steep',
 'rise',
 'govern',
 'steer',
 'lend',
 'still',
 'administr',
 'made',
 'small',
 'chang',
 'hint',
 'mr',
 'xi',
 'may',
 'focus',
 'eco

In [43]:
#
# get_avg_size_cols(t_df, ["s2"])

#Array(my_sample)



In [44]:
t_df.select("s2").show()



+--------------------+
|                  s2|
+--------------------+
|[china, economic,...|
|[trade, big, topi...|
|[top, beaches, wo...|
|[sheriff, report,...|
|[surgeon, claimin...|
|[differently, pri...|
|[samantha, bee, j...|
|[krauthammer, syr...|
|[rust, belt, vote...|
|[illegal, outpost...|
|[, 94, 4m, 198m, ...|
|[trump, mar, lago...|
|[stocks, slip, en...|
|[jared, kushner, ...|
|[cheated, ex, gir...|
|[fabulous, life, ...|
|[destinations, in...|
|[president, donal...|
|[4, things, re, p...|
|[freddie, flintof...|
+--------------------+
only showing top 20 rows



In [45]:
#t_df.distinct()<< distinct analysis of unigrams vs trigrams?
## 
t_df.select("s2").distinct().show()

+--------------------+
|                  s2|
+--------------------+
|[channelid, chann...|
|[five, year, old,...|
|[8, year, old, tr...|
|[tough, choices, ...|
|[50, cent, mercil...|
|[thane, call, cen...|
|[pegavoice, disru...|
|[another, kerala,...|
|[tech, companies,...|
|[bride, calls, we...|
|[smith, trying, s...|
|[, coup, north, c...|
|[st, louis, man, ...|
|[nike, re, releas...|
|[famous, mary, ty...|
|[dad, dresses, li...|
|[unnecessary, swe...|
|[people, rushing,...|
|[donald, trump, s...|
|[video, venture, ...|
+--------------------+
only showing top 20 rows



In [46]:
t_df=t_df.withColumn("s3",stem_lem("s2"))

In [48]:
t_df.select("s3").take(3)

[Row(s3=['china', 'econom', 'reform', 'xi', 'jinp', 'track', 'record', 'economist', 'general', 'agre', 'china', 'must', 'overhaul', 'huge', 'wast', 'economi', 'want', 'continu', 'grow', 'year', 'come', 'mean', 'limit', 'polit', 'interfer', 'bank', 'financi', 'system', 'make', 'bloat', 'industri', 'sensit', 'market', 'forc', 'lower', 'barrier', 'foreign', 'trade', 'invest', 'propon', 'took', 'heart', 'late', '2012', 'presid', 'xi', 'jinp', 'took', 'formal', 'control', 'chine', 'communist', 'parti', 'pledg', 'crack', 'corrupt', 'cut', 'red', 'tape', 'today', 'mr', 'xi', 'near', 'end', 'first', 'five', 'year', 'term', 'much', 'optim', 'among', 'economist', 'fade', 'china', 'remain', 'heavili', 'depend', 'larg', 'invest', 'maintain', 'brisk', 'slow', 'econom', 'growth', 'steep', 'rise', 'govern', 'steer', 'lend', 'still', 'administr', 'made', 'small', 'chang', 'hint', 'mr', 'xi', 'may', 'focus', 'econom', 'overhaul', 'start', 'second', 'term', 'mr', 'xi', 'done', 'far', 'import', 'done', '

In [49]:
rdd0 = t_df.select("s3").alias("s3").rdd.map(list)

In [50]:
rdd0.take(1)

[[['china',
   'econom',
   'reform',
   'xi',
   'jinp',
   'track',
   'record',
   'economist',
   'general',
   'agre',
   'china',
   'must',
   'overhaul',
   'huge',
   'wast',
   'economi',
   'want',
   'continu',
   'grow',
   'year',
   'come',
   'mean',
   'limit',
   'polit',
   'interfer',
   'bank',
   'financi',
   'system',
   'make',
   'bloat',
   'industri',
   'sensit',
   'market',
   'forc',
   'lower',
   'barrier',
   'foreign',
   'trade',
   'invest',
   'propon',
   'took',
   'heart',
   'late',
   '2012',
   'presid',
   'xi',
   'jinp',
   'took',
   'formal',
   'control',
   'chine',
   'communist',
   'parti',
   'pledg',
   'crack',
   'corrupt',
   'cut',
   'red',
   'tape',
   'today',
   'mr',
   'xi',
   'near',
   'end',
   'first',
   'five',
   'year',
   'term',
   'much',
   'optim',
   'among',
   'economist',
   'fade',
   'china',
   'remain',
   'heavili',
   'depend',
   'larg',
   'invest',
   'maintain',
   'brisk',
   'slow',
   'ec

In [51]:
# rdd1=rdd0.flatMap(lambda word: (word, 1)) \
#              .reduceByKey(lambda a, b: a + b)


In [52]:
# rdd1.take(1)
# strip_func_str(strip_possessive_str(
#     concat(col('title'),col("text")
#           ))
               

#t_df = t_df.withColumn("s3", stem_lem(lit(col("s2"))))
# #t_df.persist()
# t_df.show("s3")

In [53]:
# t_df.select('sss', "ssss").limit(5).show(truncate=False)

In [54]:
from pyspark.ml import Pipeline

ngram_pipeline=Pipeline(stages=[NGram(n=i, inputCol="s3", outputCol= str(i)+"gram") for i in range(2,4)])
m = ngram_pipeline.fit(t_df)
t_df = m.transform(t_df)
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s3: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3gram: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [55]:
# NGram Creation
"""
plan would be to turn this into a Pipeline([Tokenizer,StopWordsRemover,Ngram1,Ngram2...])
"""
if 0:
    raise AssertionError('stop')


    # This should be a pipeline
    text_cols= ["s3"]
    for t_c in text_cols:
        for i in range(1,4):
            t_df= NGram(n=i, inputCol=t_c, outputCol=t_c+"_"+str(i)+"gram").transform(t_df)
    t_df.persist()

In [56]:
#t_df.select("s3_1gram", "s").show(truncate=True)

In [57]:
#https://stackoverflow.com/questions/38839924/how-to-combine-n-grams-into-one-vocabulary-in-spark

# from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
# from pyspark.ml import Pipeline


# def build_ngrams(inputCol="tokens", n=3):

#     ngrams = [
#         NGram(n=i, inputCol="tokens", outputCol="{0}_grams".format(i))
#         for i in range(1, n + 1)
#     ]

#     vectorizers = [
#         CountVectorizer(inputCol="{0}_grams".format(i),
#             outputCol="{0}_counts".format(i))
#         for i in range(1, n + 1)
#     ]

#     assembler = [VectorAssembler(
#         inputCols=["{0}_counts".format(i) for i in range(1, n + 1)],
#         outputCol="features"
#     )]

#     return Pipeline(stages=ngrams + vectorizers + assembler)







# # Preprocessing Strategy
# text_cols= ["text","title"]

# for t_c in text_cols:
#     t_df = Tokenizer(inputCol = t_c, outputCol = "token_"+t_c).transform(t_df)
#     t_df = StopWordsRemover(inputCol = "token_"+t_c, outputCol = t_c+"_f_token").transform(t_df)
    
#     for i in range(1,4):
#         t_df= NGram(n=i, inputCol=t_c+"_f_token", outputCol=t_c+"_ngram_"+str(i)).transform(t_df)
        
#         #CountVectorizer for each ngram col
#         c_v = CountVectorizer(inputCol= t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"cvfeat",
#                               minDF=2.0, vocabSize=2**14)
        
#         #indexer = StringIndexer(inputCol=t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"ix_feat")
#         #display(t_c+"_ngram_"+str(i), t_c+"_ngram_"+str(i)+"cvfeat")
        
#         #m_ = c_v.fit(t_df) # c_v.fit is failing
#         #t_df = m_.tranform(t_df)
        

#     t_df = t_df.drop("token_"+t_c)


# #t_df.show()


# # def build_ngrams_wocs(inputCol=["text","target"], n=3):
# #     tokenizer = [Tokenizer(inputCol="text", outputCol="words")]
# #     ngrams = [
# #         NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
# #         for i in range(1, n + 1)
# #     ]

# #     cv = [
# #         CountVectorizer(vocabSize=5460,inputCol="{0}_grams".format(i),
# #             outputCol="{0}_tf".format(i))
# #         for i in range(1, n + 1)
# #     ]
# #     idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

# #     assembler = [VectorAssembler(
# #         inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
# #         outputCol="features"
# #     )]
# #     label_stringIdx = [StringIndexer(inputCol = "target", outputCol = "label")]
# #     lr = [LogisticRegression(maxIter=100)]
# #     return Pipeline(stages=tokenizer + ngrams + cv + idf+ assembler + label_stringIdx+lr)



In [58]:
# show each token
# show me grams
# show me bigrams
# show me trigrams



In [59]:
# t_df.printSchema()

In [60]:
## Before getting into trigrams maybe we can count tokens.

In [61]:
# df = spark.createDataFrame(
#     [(0, ["a", "b a", "c"]), (1, ["a", "b", "b", "c", "a"])],
#     ["label", "raw"])
# cv = CountVectorizer(inputCol="raw", outputCol="vectors")
# model = cv.fit(df)
# model.transform(df).show(truncate=False)

In [62]:
# .
# : org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 22, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
# 	at java.util.Arrays.copyOf(Arrays.java:3236)
# 	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
t_df.printSchema(), t_df.columns

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s3: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3gram: array (nullable = true)
 |    |-- element: string (containsNull = false)



(None,
 ['id', 'title', 'text', 'label', 's', 's1', 's2', 's3', '2gram', '3gram'])

In [63]:
#text_tokenizer = Tokenizer(inputCol = 'text', outputCol = 'text_words')
#ttl_tokenizer = Tokenizer(inputCol = 'title', outputCol = 'ttl_words')
#regex_tokenizer = RegexTokenizer(inputCol= 'sentence', outputCol= 'words', pattern='\\W')
#t_df = tokenizer.transform(t_df)
# t_df = text_tokenizer.transform(ttl_tokenizer.transform(t_df))
help(CountVectorizer)

Help on class CountVectorizer in module pyspark.ml.feature:

class CountVectorizer(pyspark.ml.wrapper.JavaEstimator, _CountVectorizerParams, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 |  Extracts a vocabulary from document collections and generates a :py:attr:`CountVectorizerModel`.
 |  
 |  >>> df = spark.createDataFrame(
 |  ...    [(0, ["a", "b", "c"]), (1, ["a", "b", "b", "c", "a"])],
 |  ...    ["label", "raw"])
 |  >>> cv = CountVectorizer(inputCol="raw", outputCol="vectors")
 |  >>> model = cv.fit(df)
 |  >>> model.transform(df).show(truncate=False)
 |  +-----+---------------+-------------------------+
 |  |label|raw            |vectors                  |
 |  +-----+---------------+-------------------------+
 |  |0    |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
 |  |1    |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
 |  +-----+---------------+-------------------------+
 |  ...
 |  >>> sorted(model.vocabulary) == ['a', 'b', 'c']
 |  True
 |  >>> countVectorize

In [64]:
## Feature building

# fcols = [
# #  'text_f_token',
#   'text_ngram_1',
#   'text_ngram_2',
#   'text_ngram_3',
# #  'title_f_token',
#   'title_ngram_1',
#   'title_ngram_2',
#   'title_ngram_3',
# ]



# cv = CountVectorizer(inputCol="text_ngram_2", outputCol="features")
# t_df=cv.fit(t_df).transform(t_df)




# ohe = OneHotEncoderEstimator(
#     inputCols= fcols,
#     outputCols=["feat_"+s for s in fcols])

#f_df=ohe.fit(t_df).transform(t_df)
    
#f_df=cv.fit(t_df).transform(t_df)
#OneHotEncoderEstimator
#hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [65]:
# idf= IDF(inputCol='rawFeatures', outputCol='features')
# idf_model = idf.fit(featurized_df)
# rescaled_df = idf_model.transform(featurized_df)

# rescaled_df.show(truncate=False)

t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s3: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3gram: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [66]:
count_vects = [CountVectorizer(inputCol= i_, outputCol= i_+"_cv",
                     minDF=2.0, vocabSize=800) for i_ in ["s3", '2gram', '3gram']]

In [67]:
# c_v = CountVectorizer(inputCol="s3", outputCol="s3_cv",
#                      minDF=2.0, vocabSize=800)

c_v = CountVectorizer(inputCol="3gram", outputCol="3gram_cv",
                     minDF=2.0, vocabSize=800)

#https://stackoverflow.com/questions/38839924/how-to-combine-n-grams-into-one-vocabulary-in-spark

# _p = {'inputCol':"title_ngram_1", 'outputCol':"title_ngram_1_cv",
#       'minDF':2.0, 'vocabSize':2**14}

#m_ = c_v.fit(t_df, params={'inputCol': 'text_ngram_1'})

mdl = c_v.fit(t_df)

t_df=mdl.transform(t_df)
t_df.persist()

DataFrame[id: string, title: string, text: string, label: string, s: string, s1: array<string>, s2: array<string>, s3: array<string>, 2gram: array<string>, 3gram: array<string>, 3gram_cv: vector]

In [68]:
help(NaiveBayes)

Help on class NaiveBayes in module pyspark.ml.classification:

class NaiveBayes(pyspark.ml.wrapper.JavaEstimator, pyspark.ml.param.shared.HasFeaturesCol, pyspark.ml.param.shared.HasLabelCol, pyspark.ml.param.shared.HasPredictionCol, pyspark.ml.param.shared.HasProbabilityCol, pyspark.ml.param.shared.HasRawPredictionCol, pyspark.ml.param.shared.HasThresholds, pyspark.ml.param.shared.HasWeightCol, pyspark.ml.util.JavaMLWritable, pyspark.ml.util.JavaMLReadable)
 |  Naive Bayes Classifiers.
 |  It supports both Multinomial and Bernoulli NB. `Multinomial NB
 |  <http://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html>`_
 |  can handle finitely supported discrete data. For example, by converting documents into
 |  TF-IDF vectors, it can be used for document classification. By making every vector a
 |  binary (0/1) data, it can also be used as `Bernoulli NB
 |  <http://nlp.stanford.edu/IR-book/html/htmledition/the-bernoulli-model-1.html>`_.
 |  The input feature

In [70]:
indexer = StringIndexer(inputCol="label", outputCol="label_index")
t_df = indexer.fit(t_df).transform(t_df)
t_df.persist()


# ['id',
#  'title',
#  'text',
#  'label',
#  's',
#  'ss',
#  's1',
#  'sss',
#  's2',
#  's3',
#  's2_1gram',
#  's2_2gram',
#  's2_3gram',
#  's3_1gram',
#  's3_2gram',
#  's3_3gram',
#  's3_cv',
#  'label_index']

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:32797)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:32797)

In [ ]:
PREPROCESSED_DF_CSV_PATH="file:///home/hadoop/cb/pre_train.csv"

#t_df.repartition(1).write.format('com.databricks.spark.csv').save(PREPROCESSED_DF_CSV_PATH,header = 'true')

#t_df.repartition(1).write.json(PREPROCESSED_DF_CSV_PATH,mode="overwrite")

t_df.schema

In [ ]:
t_df.columns

In [ ]:
columns_to_drop=[
#'id',
 'title',
 'text',
 'label',
 #'s',
 'ss',
 's1',
 'sss',
 's2',
# 's3',
 's2_1gram',
 's2_2gram',
#  's2_3gram',
#  's3_1gram',
#  's3_2gram',
#  's3_3gram',
#  's3_cv',
#  'label_index'
]

In [ ]:
#t_df = t_df.drop(*columns_to_drop)

t_df.columns

In [ ]:
help(t_df.write.json)

In [ ]:
#raise Exception("fail")

train, test = t_df.randomSplit([.8,.2], seed = 999)



In [ ]:
NB = NaiveBayes(modelType= "multinomial", labelCol="label_index", featuresCol ="3gram_cv")
#NB = NaiveBayes(modelType= "bernoulli", labelCol="label_index", featuresCol ="3gram_feats")



In [ ]:

nb_model= NB.fit(train)
nb_predictions= nb_model.transform(test)


"""
Py4JJavaError: An error occurred while calling o459.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 23, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
"""

In [ ]:

evaluator = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "prediction", 
                                              metricName="accuracy")

nb_accuracy = evaluator.evaluate(nb_predictions)

print("Accuracy : ", nb_accuracy)

In [ ]:
"""
https://spark.apache.org/docs/2.2.0/ml-pipeline.html

Plan:
    - use Pipeline([transformer_n, estimator_n...])
    - Improve preprocessing (punctuation, special characters, stemming)
    - Include transformers and estimators for each n_gram length

"""

In [ ]:
ml_pipeline = Pipeline(stages=[NB,nb_predictions])

# Fit the pipeline to training documents.
model = pipeline.fit(training)
prediction = model.transform(test)